In [ ]:
import sccloud as scc
import numpy as np
import pandas as pd
import bokeh

## Count Matrix File

For this tutorial, we provide a count matrix dataset on Human Bone Marrow with 8 donors. It is stored in scCloud hdf5 format (with file extension ".h5sc").

Terra Notebook uses Google Cloud SDK command to retrieve the data from a remote Google Bucket.

In [ ]:
src_file = "MantonBM_nonmix_subset.h5sc"
gf_src = "gs://fc-a40546ad-5019-43b9-b4de-5951d1e65948/" + src_file

In [ ]:
!gsutil cp $gf_src .

When the downloading is finished, you can load the file using scCloudPy `read_input` function:

In [ ]:
adata = scc.read_input(src_file)
adata

The count matrix is manipulated as an Annotated Data Matrix structure (see [anndata.AnnData](https://anndata.readthedocs.io/en/latest/anndata.AnnData.html) for details; the figure below is also borrowed from this link):

<img src="https://falexwolf.de/img/scanpy/anndata.svg" width="40%" />

It has 5 major parts:
* Raw count matrix: `adata.X`, a [Scipy sparse matrix](https://docs.scipy.org/doc/scipy/reference/generated/scipy.sparse.csr_matrix.html), with rows the cell barcodes, columns the genes/features:

In [ ]:
adata.X

This dataset contains $48,000$ barcodes and $33,694$ genes.

* Cell barcode attributes: `adata.obs`, a [Pandas data frame](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.html) with barcode as the index. For now, there is only one attribute `"Channel"` referring to the donor from which the cell comes from:

In [ ]:
adata.obs.head()

In [ ]:
adata.obs['Channel'].value_counts()

From the summary above, you can see that each of the 8 donors (numbered from 1 to 8) gives $6,000$ cells.

* Gene attributes: `adata.var`, also a Pandas data frame, with gene name as the index. For now, it only has one attribute `"gene_ids"` referring to the unique gene ID in the experiment:

In [ ]:
adata.var.head()

* Unstructured information: `adata.uns`, a Python [hashed dictionary](https://docs.python.org/3/library/collections.html#collections.OrderedDict). It usually stores information not restricted to barcodes or features, but about the whole dataset, such as its genome reference:

In [ ]:
adata.uns['genome']

* Finally, embedding attributes on cell barcodes: `adata.obsm`; as well as on genes, `adata.varm`. We'll see it in later sections.

## Preprocessing

### Filtration

The first step in preprocessing is to perform the quality control analysis, and remove cells and genes of low quality.

We can generate QC metrics using the following method with default settings:

In [ ]:
scc.qc_metrics(adata)

The metrics considered are:
* Number of genes: by default, keep cells with $500 \leq \text{# Genes} < 6000$;
* Number of UMIs: by default, keep cells with $100 \leq \text{# UMIs} < 600,000$;
* Percent of Mitochondrial genes: by default, keep cells with percent $< 10\%$;
* Percent of cells: by default, genes that are expressed in at least $0.05\%$ of cells are marked as `robust`.

For details on customizing your own thresholds, see [documentation](https://sccloudpy.readthedocs.io/en/latest/api/sccloud.qc_metrics.html).

Numeric summaries on filtration on cell barcodes and genes can be achieved by `get_filter_stats` method:

In [ ]:
stats_samples, stats_genes = scc.get_filter_stats(adata)

The results are two pandas data frames, one for samples, the other for genes:

In [ ]:
print("Filtration stats w.r.t. samples:")
stats_samples

In [ ]:
print("Filtration stats w.r.t. genes:")
stats_genes

You can also check the QC stats via plots. First is the violin plot:

In [ ]:
scc.violin(adata, keys = ['n_genes', 'n_counts', 'percent_mito'], by = 'passed_qc')

Cells marked with `passed_qc == True` are those passing QC, while the others will be filtered out.

Then is scatterplot on the QC metrics on samples:

In [ ]:
scc.scatter(adata, 'n_genes', 'n_counts', color = 'passed_qc')

This plot indicates that number of UMIs and number of genes are positively correlated.

In [ ]:
scc.scatter(adata, 'n_genes', 'percent_mito', color = 'passed_qc')

The plot above indicates that cells with high percent of mitochondrial genes tend to have fewer signals.

Now filter data based on QC metrics set in `qc_metrics`:

In [ ]:
scc.filter_data(adata)
adata

After filtration, $34,654$ cells ($72.20\%$) and $23,460$ genes ($69.63\%$) are kept.

We can also view the cells after filtration w.r.t. donors:

In [ ]:
adata.obs['Channel'].value_counts()

### Normalization and Logarithmic Transformation

After filtration, we need to first normalize the distribution of cells w.r.t. each gene to have the same count (default is $10^5$, see [documentation](https://sccloudpy.readthedocs.io/en/latest/api/sccloud.log_norm.html)), and then transform into logarithmic space by $log(x + 1)$ to avoid number explosion:

In [ ]:
scc.log_norm(adata)

For the downstream analysis, we may need to make a copy of the count matrix, in case of coming back to this step and redo the analysis:

In [ ]:
adata_trial = adata.copy()

### Highly Variable Gene Selection

**Highly Variable Genes (HVG)** are more likely to convey information discriminating different cell types and states.
Thus, rather than considering all genes, people usually focus on selected HVGs for downstream analyses.

You need to set `consider_batch` flag to consider or not consider batch effect. At this step, set it to `False`:

In [ ]:
scc.highly_variable_features(adata_trial, consider_batch = False)

By default, we select 2000 HVGs using the scCloud selection method. Alternative, you can also choose the traditional method that both *Seurat* and *SCANPY* use, by setting `flavor == 'Seurat'`. See [documentation](https://sccloudpy.readthedocs.io/en/latest/api/sccloud.highly_variable_features.html) for details.

Below is a scatterplot of highly variable genes for this dataset. Each point stands for one gene. Orange points are selected to be highly variable genes, which account for the majority of variation of the dataset.

In [ ]:
scc.variable_feature_plot(adata_trial)

Below lists the top 5 HVGs:

In [ ]:
adata_trial.var.loc[adata_trial.var['highly_variable_features']].sort_values(by = 'hvf_rank').head()

## Principal Component Analysis

To reduce the dimension of data, Principal Component Analysis (PCA) is widely used. Briefly speaking, PCA transforms the data from original dimensions into a new set of Principal Components (PC) of a much smaller size. In the transformed data, dimension is reduced, while PCs still cover a majority of the variation of data. Moreover, the new dimensions (i.e. PCs) are independent with each other.

`scCloudPy` uses the following method to perform PCA:

In [ ]:
scc.pca(adata_trial)

By default, `scc.pca` uses:
* Before PCA, scale the data to standard Normal distribution $N(0, 1)$, and truncate them with max value $10$;
* Number of PCs to compute: 50;
* Apply PCA only to highly variable features.

See [its documentation](https://sccloudpy.readthedocs.io/en/latest/api/sccloud.pca.html) for customization.

To explain the meaning of PCs, let's look at the first PC (denoted as $PC_1$), which covers the most of variation:

In [ ]:
coord_pc1 = adata_trial.uns['PCs'][:, 0]
coord_pc1

This is an array of 2000 elements, each of which is a coefficient corresponding to one HVG.

With the HVGs as the following:

In [ ]:
adata_trial.var.loc[adata_trial.var['highly_variable_features']].index.values

$PC_1$ is computed by

\begin{equation*}
PC_1 = \text{coord_pc1}[0] \cdot \text{HES4} + \text{coord_pc1}[1] \cdot \text{ISG15} + \text{coord_pc1}[2] \cdot \text{TNFRSF18} + \cdots + \text{coord_pc1}[1997] \cdot \text{S100B} + \text{coord_pc1}[1998] \cdot \text{MT-CO1} + \text{coord_pc1}[1999] \cdot \text{MT-CO3}
\end{equation*}

Therefore, all the 50 PCs are the linear combinations of the 2000 HVGs.

The calculated PCA count matrix is stored in the `obsm` field, which is the first embedding object we have 

In [ ]:
adata_trial.obsm['X_pca'].shape

For each of the $34,654$ cells, its count is now w.r.t. 50 PCs, instead of 2000 HVGs.

## Nearest Neighbors

All the downstream analysis, including clustering and visualization, needs to construct a k-Nearest-Neighbor (kNN) graph on cells. We can build such a graph using `neighbors` method:

In [ ]:
scc.neighbors(adata_trial)

It uses the default setting:
* For each cell, calculate its 100 nearest neighbors;
* Use PCA matrix for calculation;
* Use L2 distance as the metric;
* Use [hnswlib](https://github.com/nmslib/hnswlib) search algorithm to calculate the approximated nearest neighbors in a really short time.

See [its documentation](https://sccloudpy.readthedocs.io/en/latest/api/sccloud.neighbors.html) for customization.

Below is the result:

In [ ]:
print("Get {} nearest neighbors (excluding itself) for each cell.".format(adata_trial.uns['pca_knn_indices'].shape[1]))
adata_trial.uns['pca_knn_indices']

In [ ]:
adata_trial.uns['pca_knn_distances']

Each row corresponds to one cell, listing its neighbors (not including itself) from nearest to farthest. `adata_trial.uns['pca_knn_indices']` stores their indices, and `adata_trial.uns['pca_knn_distances']` stores distances.

## Clustering and Visualization

Now we are ready to cluster the data for cell type detection. `scCloudPy` provides 4 clustering algorithms to use. In this tutorial, we use the Louvain algorithm:

In [ ]:
scc.louvain(adata_trial)

As a result, Louvain algorithm finds 19 clusters:

In [ ]:
adata_trial.obs['louvain_labels'].value_counts()

We can check each cluster's composition regarding donors via a composition plot:

In [ ]:
scc.composition_plot(adata_trial, by = 'louvain_labels', condition = 'Channel')

However, we can see a clear batch effect in the plot: e.g. Cluster 10 and 13 have most cells from Donor 3.

We can see it more clearly in its FIt-SNE plot (a visualization algorithm which we will talk about later):

In [ ]:
scc.fitsne(adata_trial)
scc.embedding(adata_trial, basis = 'fitsne', keys = ['louvain_labels', 'Channel'],
              size=1, alpha=0.8, color=bokeh.palettes.all_palettes['Category20'][20])

## Batch Correction

Batch effect occurs when data samples are generated in different conditions, such as date, weather, lab setting, equipment, etc. Unless informed that all the samples were generated under the similar condition, people may suspect presumable batch effects if they see a visualization graph with samples kind-of isolated from each other.

For this dataset, we need the batch correction step to reduce such a batch effect, which is observed in the plot above:

In [ ]:
scc.highly_variable_features(adata, consider_batch = True)
scc.correct_batch(adata, features = 'highly_variable_features')

In [ ]:
adata.uns['fmat_highly_variable_features'].shape

Batch correction result is stored at `adata.uns['fmt_highly_variable_features']`. By default, it uses `adata.obs['Channel']` as the batch key, i.e. there are 8 batches in this dataset.

See [its documnetation](https://sccloudpy.readthedocs.io/en/latest/api/sccloud.correct_batch.html) for customization. Moreover, if you want to use some other cell attribute(s) as the batch key, consider using `scc.set_group_attribute` method ([see details](https://sccloudpy.readthedocs.io/en/latest/api/sccloud.set_group_attribute.html)).

## Repeat Previous Steps on the Corrected Data

As the count matrix is changed by batch correction, we need to redo steps starting from PCA down to clustering:

In [ ]:
scc.pca(adata)
scc.neighbors(adata)
scc.louvain(adata)

Let's check the composition plot now:

In [ ]:
scc.composition_plot(adata, by = 'louvain_labels', condition = 'Channel')

If everything goes properly, you should be able to see that no cluster has a dominant donor cells. Also notice that Louvain algorithm on the corrected data finds 16 clusters, instead of the original 19 ones.

Also, FIt-SNE plot is different:

In [ ]:
scc.fitsne(adata)
scc.embedding(adata, basis = 'fitsne', keys = ['louvain_labels', 'Channel'],
              size=1, alpha=0.8, color=bokeh.palettes.all_palettes['Category20'][20])

Similarly, if everything goes properly, you should be able to see that cells from different donors are better mixed in the right-hand-side plot.

## Visualization

### tSNE Plot

In previous sections, we have seen data visualization using FIt-SNE. FIt-SNE is a fast implementation on tSNE algorithm. Including FIt-SNE, `scCloudPy` provides 3 different tSNE plotting methods:

* `scc.fitsne`: FIt-SNE plot, using [fitsne](https://github.com/KlugerLab/FIt-SNE) package. [See details](https://sccloudpy.readthedocs.io/en/latest/api/sccloud.fitsne.html)
* `scc.tsne`: tSNE plot, using [Multicore-TSNE](https://github.com/DmitryUlyanov/Multicore-TSNE) package. [See details](https://sccloudpy.readthedocs.io/en/latest/api/sccloud.tsne.html)
* `scc.net_tsne`: approximated tSNE plot with speed up using Deep Neural Networks (DNN) models. [See details](https://sccloudpy.readthedocs.io/en/latest/api/sccloud.net_tsne.html)

### UMAP Plot

Besides tSNE, `scCloudPy` also provides UMAP plotting methods:

* `scc.umap`: UMAP plot, using [umap-learn](https://github.com/lmcinnes/umap) package. [See details](https://sccloudpy.readthedocs.io/en/latest/api/sccloud.umap.html)
* `scc.net_umap`: Approximated UMAP plot with DNN model based speed up. [See details](https://sccloudpy.readthedocs.io/en/latest/api/sccloud.net_umap.html)

Below is the UMAP plot of the data using `umap` method:

In [ ]:
scc.umap(adata)
scc.embedding(adata, basis = 'umap', keys = ['louvain_labels', 'Channel'], 
              size = 1, alpha = 0.8, color = bokeh.palettes.all_palettes['Category20'][20])

## Differential Expression Analysis

With the clusters ready, we can now perform Differential Expression (DE) Analysis, which is crucial for identifying the cell type of each cluster.

Now use `de_analysis` method to run DE analysis. We use Louvain result here. 

Notice that for notebooks running on Terra, you need to explicitly set `temp_folder` parameter to avoid the incorrect "No space left" error:

In [ ]:
scc.de_analysis(adata, cluster = 'louvain_labels', auc = False, t = True, fisher = False, mwu = False,
                temp_folder = "/tmp")

By default, DE analysis runs 
* `auc`: Area under ROC (AUROC) and Area under Precision-Recall (AUPR).
* `t`: Welch’s t test.

Alternatively, you can also run the follow tests by setting their corresponding parameters to be `True`:
* `fisher`: Fisher’s exact test.
* `mwu`: Mann-Whitney U test.

DE analysis result is stored with key `de_res` (by default) in `varm` field of data. See [documentation](https://sccloudpy.readthedocs.io/en/latest/api/sccloud.de_analysis.html) for more details. 

To load the result in a human-readable format, use `markers` method:

In [ ]:
marker_dict = scc.markers(adata)

By default, `markers`:
* Sort genes by WAD scores in descending order;
* Use $\alpha = 0.05$ significance level on q-values for inference.

See [documentation](https://sccloudpy.readthedocs.io/en/latest/api/sccloud.markers.html) for customizing these parameters.

Let's see the up-regulated genes for Cluster 1:

In [ ]:
marker_dict['1']['up']

Among them, **TRAC** worth notification. It is a critical marker for T cells.

We can also use Volcano plot to see the DE result. Below is such a plot w.r.t. Cluster 1 with log fold change being the metric:

In [ ]:
scc.volcano(adata, cluster_ids = ['1'])

To store a specific DE analysis result to file, you can `write_results_to_excel` methods in `scCloud`:

In [ ]:
#scc.write_results_to_excel(marker_dict, "MantonBM_nonmix_subset.louvain_labels.de.xlsx")

## Cell Type Annotation

After done with DE analysis, we can use the test result to annotate the clusters.

In [ ]:
celltype_dict = scc.infer_cell_types(adata, markers = 'human_immune', de_test = 't')
cluster_names = scc.infer_cluster_names(celltype_dict)

`infer_cell_types` has 2 critical parameters to set:
* `markers`: Either `'human_immune'`, `'mouse_immune'`, `'human_brain'`, `'mouse_brain'`, or a user-specified marker dictionary.
* `de_test`: Decide which DE analysis test to be used for cell type inference. It can be either `'t'`, `'fisher'`, or `'mwu'`.

`infer_cluster_names` by default uses `threshold = 0.5` to filter out candidate cell types of scores lower than 0.5.

See [documentation](https://sccloudpy.readthedocs.io/en/latest/api/sccloud.infer_cell_types.html) for details.

Next, substitute the inferred cluster names in data:

In [ ]:
adata.rename_categories('louvain_labels', cluster_names)

And plot data with cluster names. You can either plot names on data or not, by setting `labels_on_data` flag:

In [ ]:
scc.embedding(adata, basis = 'fitsne', keys = ['louvain_labels'], labels_on_data = False,
              padding = (0.1, 0.01), width = 600, size = 1, alpha = 0.8,
              color = bokeh.palettes.all_palettes['Category20'][20])

In [ ]:
scc.embedding(adata, basis = 'fitsne', keys = ['louvain_labels'], labels_on_data = True,
              padding = (0.1, 0.01), width = 600, size = 1, alpha = 0.8,
              color = bokeh.palettes.all_palettes['Category20'][20])

## Gene-specific Plots

### Dot Plot

In [ ]:
marker_genes = ['CD38', 'JCHAIN', 'FCGR3A', 'HLA-DPA1', 'CD14', 'CD79A', 'MS4A1', 'CD34', 'TRAC', 'CD3D', 'CD8A',
                'CD8B', 'GYPA', 'NKG7', 'CD4', 'SELL', 'CCR7']

scc.dotplot(adata, keys = marker_genes, by = 'louvain_labels')

### Heatmap

In [ ]:
scc.heatmap(adata, keys = marker_genes, by = 'louvain_labels')

### Violin Plot

In [ ]:
scc.violin(adata, keys = ['TRAC'], by = 'louvain_labels', width = 900, height = 450)

### Feature Plot

In [ ]:
scc.embedding(adata, basis = 'fitsne', keys = ['TRAC', 'CD79A', 'CD14', 'CD34'])

## Cell Development Trajectory and Diffusion Map

Alternative, `scCloudPy` provides cell development trajectory plots using Force-directed Layout (FLE) algorithm:
* `scc.fle`: FLE plot, using Force-Atlas 2 algorithm in [forceatlas2-python](https://github.com/klarman-cell-observatory/forceatlas2-python) package. [See details](https://sccloudpy.readthedocs.io/en/latest/api/sccloud.fle.html)
* `scc.net_fle`: Approximated FLE plot with DNN model based speed up. [See details](https://sccloudpy.readthedocs.io/en/latest/api/sccloud.net_fle.html)

Moreover, calculation of FLE plots is on Diffusion Map of the data, rather than directly on data points, in order to achieve a better efficiency. Thus, we need to first compute the diffusion map structure:

In [ ]:
scc.diffmap(adata)

By default, `diffmap` method uses:
* Number of Diffusion Components = $100$.
* Compute diffusion map from PCA matrix.

In [ ]:
adata.obsm['X_diffmap'].shape

Now we are ready to calculate the cell developing trajectory. Below is FLE plot using `net_fle` method:

In [ ]:
scc.net_fle(adata)
scc.embedding(adata, basis = 'net_fle', keys = ['louvain_labels'],
              size = 1, alpha = 0.8, color = bokeh.palettes.all_palettes['Category20'][20],
              width = 700, height = 700)

## (Advanced) More on Clustering

In previous sections, we use Louvain algorithm for clustering. Including Louvain, `scCloudPy` provides 4 algorithms:
* `scc.louvain`: Louvain algorithm, using [louvain-igraph](https://github.com/vtraag/louvain-igraph) package.
* `scc.leiden`: Leiden algorithm, using [leidenalg](https://github.com/vtraag/leidenalg) package.
* `scc.spectral_louvain`: Spectral Louvain algorithm, which requires Diffusion Map.
* `scc.spectral_leiden`: Spectral Leiden algorithm, which requires Diffusion Map.

We have used `louvain` method. Its documentation is [here](https://sccloudpy.readthedocs.io/en/latest/api/sccloud.louvain.html). By default, it uses resolution 1.3, does clustering via PCA matrix, and stores result with key `louvain_labels` in `obs` field of data.

**Leiden Algorithm.** `leiden` method ([See details](https://sccloudpy.readthedocs.io/en/latest/api/sccloud.leiden.html)), by default, uses resolution 1.3, does clustering via PCA matrix, runs the algorithm until reaching its optimal clustering (`n_iter` parameter), and stores result with key `leiden_labels` in `obs` field of data.

However, with the default `n_iter` parameter, it can take a very long time to converge, while the result doesn't improved much. So instead, you may think of feed it a small positive number to put a hard specification on how many iterations to run. Here, we use 2:

In [ ]:
scc.leiden(adata, n_iter = 2)

We can compare its result with Louvain's via UMAP:

In [ ]:
scc.embedding(adata, basis = 'umap', keys = ['louvain_labels', 'leiden_labels'],
              size = 1, alpha = 0.8, color = bokeh.palettes.all_palettes['Category20'][20])

**Spectral Louvain Algorithm** `spectral_louvain` method ([See details](https://sccloudpy.readthedocs.io/en/latest/api/sccloud.spectral_louvain.html)), by default, uses resolution 1.3, runs KMeans on Diffusion Map, does clustering on PCA matrix, and stores result with key `spectral_louvain_labels` in `obs` field of data.

In [ ]:
scc.spectral_louvain(adata)

**Spectral Leiden Algorithm** `spectral_leiden` method ([See details](https://sccloudpy.readthedocs.io/en/latest/api/sccloud.spectral_leiden.html)), by default, uses resolution 1.3, runs KMeans on Diffusion Map, does clustering on PCA matrix, and stores result with key `spectral_leiden_labels` in `obs` field of data.

In [ ]:
scc.spectral_leiden(adata)

Below are the UMAP plots of the clustering results of these two algorithms:

In [ ]:
scc.embedding(adata, basis = 'umap', keys = ['spectral_louvain_labels', 'spectral_leiden_labels'],
              size = 1, alpha = 0.8, color = bokeh.palettes.all_palettes['Category20'][20])